## Folding proteins with Cradle
This tutorial will walk you through the the simple steps you need to peform to be able to fold proteins using Cradle Bio. The setup is easy and it should take just a couple of minutes.

### Install the client library
Using pip:
```bash
pip install cradlebio
```

<details>
    <summary>+Other installation methods</summary>
    
  1. Using conda
```
conda install cradlebio
```

  2. Directly form source. If you would like to install the latest Cradle version:
```
pip install git+https://gitlab.com/cradlebio/studio/client.git
```
</details>
    
### Get an authentication token
Navigate to [https://auth.internal.cradle.bio/](https://auth.internal.cradle.bio/), sign in, and
download the credentials to a location of your choice, let's call it `CREDENTIALS_FILE`.

### Colab users
If you are running inside a Google Colab notebook, please upgrade your python version using:
```
!pip install ipython ipykernel --upgrade
```

That's it! You are now ready to fold proteins.

### Folding
Place the proteins that you would like to fold in a FASTA file, e.g.
```
>protein1
MTDDKDVLRD...
```

In [4]:
from cradlebio import alphafold, auth

# enter path to the credentials you generated at https://auth.internal.cradle.bio/ here
CREDS_FILE = '/Users/dd/cradle/client/tests/cradlebio/data/test_creds.json'  
FASTA = '/tmp/harmen.fasta'  # enter path to your FASTA file here

creds=auth.IdentityPlatformTokenCredentials.from_file(CREDS_FILE)
folder = alphafold.Alphafold(creds)
sequences = folder.predict(FASTA)

100%|██████████████████████████████████████████████████████████| 100/100 [07:33<00:00,  4.54s/%, DONE        ]


Your proteins are now folding.

The return value, `sequences`, is a list of [cradlebio.alphafold.Sequence](https://gitlab.com/cradlebio/studio/client/-/blob/main/cradlebio/alphafold.py#L39)
objects. You can get the PDB (Protein Data Bank) file that contains the actual folding for each sequence by calling the `pdb()`
method. Since folding can take about 20 minutes, `pdb()` can block for a long time:
```python
pds = seq.pdb()
```

The code snippet below iterates through all the sequences and prints the location of the alignment (a3m) and PDB files:

In [9]:
print(alphafold.get_jobs(creds))

[Job(id='2022-06-11-19:02:47_harmen.fasta', job_data={'md5sum': 'c1df16e9a0ddb7c5fb3f5b70a080b9df', 'creation_time': DatetimeWithNanoseconds(2022, 6, 11, 17, 2, 47, 403307, tzinfo=datetime.timezone.utc), 'status': 'DONE'}, sequences=[<cradlebio.alphafold.Sequence object at 0x7fe391c1fbb0>]), Job(id='2022-06-11-18:58:22_harmen.fasta', job_data={'status': 'DONE', 'md5sum': '4ff50def3bda812705f3aa2a720152b3', 'creation_time': DatetimeWithNanoseconds(2022, 6, 11, 16, 58, 22, 548843, tzinfo=datetime.timezone.utc)}, sequences=[<cradlebio.alphafold.Sequence object at 0x7fe391c03280>])]


This code snippet visualizes the folded protein. Make sure you install py3Dmol beforehand using:
```shell
conda install -c conda-forge py3dmol
```

In [ ]:
import py3Dmol
from google.cloud import storage

view=py3Dmol.view()

pdb_str = sequences[0].pdb()[0]
view.addModel(pdb_str,'pdb')
view.setStyle( {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()